import packages
------------------

Some packages are installed automatically if you use Anaconda. As pytorch is used here, you are expected to install that in your machine. 

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/UTRECHT/utils')
sys.path.insert(0,'/content/gdrive/MyDrive/UTRECHT')

import utils

In [3]:
from __future__ import print_function, division
import os, random, time, copy
from skimage import io, transform, morphology, feature
import numpy as np
import os.path as path
import scipy.io as sio
from scipy import misc
from scipy import ndimage, signal
import scipy
import pickle
import sys
import math
import matplotlib.pyplot as plt
import PIL.Image
from io import BytesIO
from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from random import sample

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler 
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms

from utils.flow_functions import *

import warnings # ignore warnings
warnings.filterwarnings("ignore")
print(sys.version)
print(torch.__version__)

3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
1.13.1+cu116


Setup config parameters
 -----------------
 
 There are several things to setup, like which GPU to use, where to read images and save files, etc. Please read and understand this. By default, you should be able to run this script smoothly by changing nothing.

In [4]:
### set path to image stacks, masks, and annotation files
path_to_image = '/content/gdrive/MyDrive/UTRECHT/Detection/Training_Tiles3'
path_to_label = '/content/gdrive/MyDrive/UTRECHT/Detection/Masks9_noNPP_circleMask'
path_to_annotCombo = '/content/gdrive/MyDrive/UTRECHT/Detection/AnnotCombo9_noNPP_circleMask'

In [5]:
#### set project name, save directory for the project
project_name = '01_Generalized_Model_SlideLvlSplit/Redo'
trial = 'Trial_01' 

exp_dir = '/content/gdrive/MyDrive/exp' # experiment directory, used for reading the init model
exp_dir = os.path.join(exp_dir, project_name)
save_dir = os.path.join(exp_dir, trial) 
if not os.path.exists(save_dir): 
    os.makedirs(save_dir)

# For Trial 1: Compile the test set by sampling from each slide.

In [6]:
#make a list of all slides
tmpSlideList =  os.listdir(path_to_label)
tmpSlideList_split = []

for i in tmpSlideList:
  j = i.split('_scan')
  tmpSlideList_split.append(j[0])

tmpSlideList = sorted(list(set(tmpSlideList_split)))

print(tmpSlideList)
print(len(tmpSlideList))

['PALIV_20-21_slide1', 'PALIV_21-22_slide 1', 'PALIV_22-23_slide 1', 'PALIV_23-24_slide 1', 'PALIV_24-25_slide 1', 'PALIV_25-26_slide 1', 'PALIV_27-28_glycerine', 'PALIV_27-28_slide 1', 'PALIV_36-37_slide 1', 'PALIV_45-46_glycerine_slide 1', 'PALIV_B_17.8_slide1', 'PALIV_B_19.8_slide2', 'PALIV_B_19_slide2', 'PALIV_B_Sample_26-27_glyc', 'PALIV_B_Sample_28-29_glyc', 'PALIV_B_Sample_31-32', 'PALIV_B_Sample_32-33_glyc', 'PALIV_B_Sample_33-34_glyc', 'PALIV_B_Sample_34-35_glyc']
19


In [7]:
# create dict of slides
slide_dict = {}
tmpSampleList=[]

for i in tmpSlideList:
  tmp_slide_list = []
  for j in os.listdir(path_to_label):
    tmpSampleList.append(j)
    if i in j:
      tmp_slide_list.append(j)

  slide_dict.update({i:tmp_slide_list})
print(slide_dict)   

{'PALIV_20-21_slide1': ['PALIV_20-21_slide1_scan2_zstack_BMP_13', 'PALIV_20-21_slide1_scan2_zstack_BMP_25', 'PALIV_20-21_slide1_scan2_zstack_BMP_38', 'PALIV_20-21_slide1_scan3_1', 'PALIV_20-21_slide1_scan3_2', 'PALIV_20-21_slide1_scan3_10', 'PALIV_20-21_slide1_scan3_3', 'PALIV_20-21_slide1_scan3_15', 'PALIV_20-21_slide1_scan3_23', 'PALIV_20-21_slide1_scan3_29', 'PALIV_20-21_slide1_scan3_30', 'PALIV_20-21_slide1_scan3_34', 'PALIV_20-21_slide1_scan3_39', 'PALIV_20-21_slide1_scan3_50', 'PALIV_20-21_slide1_scan3_52', 'PALIV_20-21_slide1_scan3_53', 'PALIV_20-21_slide1_scan3_70', 'PALIV_20-21_slide1_scan3_66', 'PALIV_20-21_slide1_scan3_71', 'PALIV_20-21_slide1_scan3_75', 'PALIV_20-21_slide1_scan3_86', 'PALIV_20-21_slide1_scan3_88', 'PALIV_20-21_slide1_scan3_95', 'PALIV_20-21_slide1_scan3_93', 'PALIV_20-21_slide1_scan3_99', 'PALIV_20-21_slide1_scan3_119', 'PALIV_20-21_slide1_scan3_113', 'PALIV_20-21_slide1_scan3_120', 'PALIV_20-21_slide1_scan3_132', 'PALIV_20-21_slide1_scan3_125', 'PALIV_20-2

In [8]:
image_stacks_per_slide = []
image_stacks_per_slide_dict = {}
for slide in slide_dict:
  image_stacks_per_slide.append(len(slide_dict[slide]))
  image_stacks_per_slide_dict.update({slide:len(slide_dict[slide])})

print(sorted(image_stacks_per_slide))
print(image_stacks_per_slide_dict)

[41, 64, 83, 101, 103, 104, 110, 113, 114, 127, 128, 142, 143, 156, 172, 238, 309, 346, 392]
{'PALIV_20-21_slide1': 110, 'PALIV_21-22_slide 1': 128, 'PALIV_22-23_slide 1': 392, 'PALIV_23-24_slide 1': 64, 'PALIV_24-25_slide 1': 346, 'PALIV_25-26_slide 1': 156, 'PALIV_27-28_glycerine': 113, 'PALIV_27-28_slide 1': 83, 'PALIV_36-37_slide 1': 104, 'PALIV_45-46_glycerine_slide 1': 101, 'PALIV_B_17.8_slide1': 238, 'PALIV_B_19.8_slide2': 114, 'PALIV_B_19_slide2': 142, 'PALIV_B_Sample_26-27_glyc': 172, 'PALIV_B_Sample_28-29_glyc': 41, 'PALIV_B_Sample_31-32': 309, 'PALIV_B_Sample_32-33_glyc': 103, 'PALIV_B_Sample_33-34_glyc': 143, 'PALIV_B_Sample_34-35_glyc': 127}


In [9]:
# FOR TRIAL 1: select three slides with approximately 125 samples each
test_det_list = []
test_slide_list = ['PALIV_20-21_slide1','PALIV_25-26_slide 1','PALIV_B_Sample_34-35_glyc']
for slide in slide_dict: 
    if slide in test_slide_list:
      tmp_sample_list = slide_dict[slide]
      for i in tmp_sample_list:
        test_det_list.append(i)
      
print(test_det_list)
print(len(test_det_list))

['PALIV_20-21_slide1_scan2_zstack_BMP_13', 'PALIV_20-21_slide1_scan2_zstack_BMP_25', 'PALIV_20-21_slide1_scan2_zstack_BMP_38', 'PALIV_20-21_slide1_scan3_1', 'PALIV_20-21_slide1_scan3_2', 'PALIV_20-21_slide1_scan3_10', 'PALIV_20-21_slide1_scan3_3', 'PALIV_20-21_slide1_scan3_15', 'PALIV_20-21_slide1_scan3_23', 'PALIV_20-21_slide1_scan3_29', 'PALIV_20-21_slide1_scan3_30', 'PALIV_20-21_slide1_scan3_34', 'PALIV_20-21_slide1_scan3_39', 'PALIV_20-21_slide1_scan3_50', 'PALIV_20-21_slide1_scan3_52', 'PALIV_20-21_slide1_scan3_53', 'PALIV_20-21_slide1_scan3_70', 'PALIV_20-21_slide1_scan3_66', 'PALIV_20-21_slide1_scan3_71', 'PALIV_20-21_slide1_scan3_75', 'PALIV_20-21_slide1_scan3_86', 'PALIV_20-21_slide1_scan3_88', 'PALIV_20-21_slide1_scan3_95', 'PALIV_20-21_slide1_scan3_93', 'PALIV_20-21_slide1_scan3_99', 'PALIV_20-21_slide1_scan3_119', 'PALIV_20-21_slide1_scan3_113', 'PALIV_20-21_slide1_scan3_120', 'PALIV_20-21_slide1_scan3_132', 'PALIV_20-21_slide1_scan3_125', 'PALIV_20-21_slide1_scan3_135', 'P

# FOR TRIAL N>1: Read the test set for Trial 1 so it stays the same across trials.


In [ ]:
trial1_save_dir = os.path.join(exp_dir, 'Trial_01')
with open(os.path.join(trial1_save_dir,'dbinfo.pkl'), 'rb') as handle:
    dbinfo = pickle.load(handle)

test_det_list = dbinfo['test_det_list']
print(len(test_det_list))

380


# Compile the training list.

In [13]:
# compile train slide list out of the remaining slides not in the test slide list
train_slide_list = []
test_slide_list = ['PALIV_20-21_slide1','PALIV_25-26_slide 1','PALIV_B_Sample_34-35_glyc']
for slide in slide_dict: 
    if slide not in test_slide_list:
      train_slide_list.append(slide)

In [14]:
# randomly select 10 of those slides as the training set.

trRatio = 0.5

train_slide_list = sample(train_slide_list,10)

train_det_list = []
for slide in slide_dict: 
    if slide in train_slide_list:
      tmp_sample_list = slide_dict[slide]
      for i in tmp_sample_list:
        train_det_list.append(i)
      
print(train_det_list)
print(len(train_det_list))

['PALIV_21-22_slide 1_scan1_13', 'PALIV_21-22_slide 1_scan1_28', 'PALIV_21-22_slide 1_scan1_14', 'PALIV_21-22_slide 1_scan1_48', 'PALIV_21-22_slide 1_scan1_52', 'PALIV_21-22_slide 1_scan1_51', 'PALIV_21-22_slide 1_scan1_61', 'PALIV_21-22_slide 1_scan1_59', 'PALIV_21-22_slide 1_scan1_65', 'PALIV_21-22_slide 1_scan1_68', 'PALIV_21-22_slide 1_scan1_67', 'PALIV_21-22_slide 1_scan1_73', 'PALIV_21-22_slide 1_scan1_72', 'PALIV_21-22_slide 1_scan1_80', 'PALIV_21-22_slide 1_scan1_90', 'PALIV_21-22_slide 1_scan1_84', 'PALIV_21-22_slide 1_scan1_101', 'PALIV_21-22_slide 1_scan1_104', 'PALIV_21-22_slide 1_scan1_105', 'PALIV_21-22_slide 1_scan1_107', 'PALIV_21-22_slide 1_scan1_110', 'PALIV_21-22_slide 1_scan1_113', 'PALIV_21-22_slide 1_scan1_115', 'PALIV_21-22_slide 1_scan1_117', 'PALIV_21-22_slide 1_scan1_128', 'PALIV_21-22_slide 1_scan1_130', 'PALIV_21-22_slide 1_scan1_131', 'PALIV_21-22_slide 1_scan1_132', 'PALIV_21-22_slide 1_scan1_135', 'PALIV_21-22_slide 1_scan1_136', 'PALIV_21-22_slide 1_scan

In [15]:
# save all the info in a pickle file to be read in the next notebook
dbinfo = {}
dbinfo['train_det_list'] = train_det_list
dbinfo['test_det_list'] = test_det_list

with open(os.path.join(save_dir,'dbinfo.pkl'), 'wb') as handle:
    pickle.dump(dbinfo, handle) # , protocol=pickle.HIGHEST_PROTOCOL

In [16]:
# see the lengths of the training and test datasets
with open(os.path.join(save_dir,'dbinfo.pkl'), 'rb') as handle:
    dbinfo = pickle.load(handle)

dbinfo.keys(), len(dbinfo['train_det_list']), len(dbinfo['test_det_list'])

(dict_keys(['train_det_list', 'test_det_list']), 1432, 393)

In [17]:
dbinfo['train_det_list']

['PALIV_21-22_slide 1_scan1_13',
 'PALIV_21-22_slide 1_scan1_28',
 'PALIV_21-22_slide 1_scan1_14',
 'PALIV_21-22_slide 1_scan1_48',
 'PALIV_21-22_slide 1_scan1_52',
 'PALIV_21-22_slide 1_scan1_51',
 'PALIV_21-22_slide 1_scan1_61',
 'PALIV_21-22_slide 1_scan1_59',
 'PALIV_21-22_slide 1_scan1_65',
 'PALIV_21-22_slide 1_scan1_68',
 'PALIV_21-22_slide 1_scan1_67',
 'PALIV_21-22_slide 1_scan1_73',
 'PALIV_21-22_slide 1_scan1_72',
 'PALIV_21-22_slide 1_scan1_80',
 'PALIV_21-22_slide 1_scan1_90',
 'PALIV_21-22_slide 1_scan1_84',
 'PALIV_21-22_slide 1_scan1_101',
 'PALIV_21-22_slide 1_scan1_104',
 'PALIV_21-22_slide 1_scan1_105',
 'PALIV_21-22_slide 1_scan1_107',
 'PALIV_21-22_slide 1_scan1_110',
 'PALIV_21-22_slide 1_scan1_113',
 'PALIV_21-22_slide 1_scan1_115',
 'PALIV_21-22_slide 1_scan1_117',
 'PALIV_21-22_slide 1_scan1_128',
 'PALIV_21-22_slide 1_scan1_130',
 'PALIV_21-22_slide 1_scan1_131',
 'PALIV_21-22_slide 1_scan1_132',
 'PALIV_21-22_slide 1_scan1_135',
 'PALIV_21-22_slide 1_scan1_13

In [18]:
dbinfo['test_det_list']

['PALIV_20-21_slide1_scan2_zstack_BMP_13',
 'PALIV_20-21_slide1_scan2_zstack_BMP_25',
 'PALIV_20-21_slide1_scan2_zstack_BMP_38',
 'PALIV_20-21_slide1_scan3_1',
 'PALIV_20-21_slide1_scan3_2',
 'PALIV_20-21_slide1_scan3_10',
 'PALIV_20-21_slide1_scan3_3',
 'PALIV_20-21_slide1_scan3_15',
 'PALIV_20-21_slide1_scan3_23',
 'PALIV_20-21_slide1_scan3_29',
 'PALIV_20-21_slide1_scan3_30',
 'PALIV_20-21_slide1_scan3_34',
 'PALIV_20-21_slide1_scan3_39',
 'PALIV_20-21_slide1_scan3_50',
 'PALIV_20-21_slide1_scan3_52',
 'PALIV_20-21_slide1_scan3_53',
 'PALIV_20-21_slide1_scan3_70',
 'PALIV_20-21_slide1_scan3_66',
 'PALIV_20-21_slide1_scan3_71',
 'PALIV_20-21_slide1_scan3_75',
 'PALIV_20-21_slide1_scan3_86',
 'PALIV_20-21_slide1_scan3_88',
 'PALIV_20-21_slide1_scan3_95',
 'PALIV_20-21_slide1_scan3_93',
 'PALIV_20-21_slide1_scan3_99',
 'PALIV_20-21_slide1_scan3_119',
 'PALIV_20-21_slide1_scan3_113',
 'PALIV_20-21_slide1_scan3_120',
 'PALIV_20-21_slide1_scan3_132',
 'PALIV_20-21_slide1_scan3_125',
 'PAL

In [ ]:
numSample2Show = 1
figWinNumHeight, figWinNumWidth = 3, 3
plt.figure(figsize=(10, 6), dpi=128, facecolor='w', edgecolor='k') # figsize -- inch-by-inch
plt.clf()
subwinCount = 1    


plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
subwinCount += 1
plt.imshow(image)
#plt.axis('off')
plt.title('image')


plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
subwinCount += 1
plt.imshow(label)
#plt.axis('off')
plt.title('label')
plt.colorbar()


plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
subwinCount += 1
plt.imshow(mask_distanceTransform)
#plt.axis('off')
plt.title('dist-transform')
plt.colorbar()


plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
subwinCount += 1
plt.imshow(mask_peaks)
#plt.axis('off')
plt.title('peaks')
plt.colorbar()


plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
subwinCount += 1
plt.imshow(mask_radius)
#plt.axis('off')
plt.title('radius')
plt.colorbar()


plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
subwinCount += 1
plt.imshow(mask_overlap)
#plt.axis('off')
plt.title('overlap')
plt.colorbar()


plt.subplot(figWinNumHeight,figWinNumWidth,subwinCount)
subwinCount += 1
flowVisShow = objDemoShowFlow.computeColor(
    mask_voteX/np.abs(mask_voteX).max(), 
    mask_voteY/np.abs(mask_voteY).max())/255.    
plt.imshow(flowVisShow)
#plt.axis('off')
plt.title('mask offset to center')
plt.colorbar()

SyntaxError: ignored

In [ ]:
type(image), image.shape, image.max(), image.min()

(numpy.ndarray, (1040, 1392, 3), 237, 0)



Leaving Blank
-----